In [2]:
import pandas as pd

user_data = pd.read_csv("all_user_data_c_50_95.csv")

In [3]:
user_data.drop(user_data.columns[[0]], axis=1, inplace=True)
user_data.head()

,id,name,editcount,status
0,15,!!,1540,0
1,228,!dea4u,6722,0
2,387,"""Country"" Bushrod Washington",1652,0
3,582,"""Yahia""",1023,0
4,829,$1LENCE D00600D,18642,0


In [4]:
user_data.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,id,editcount,status
count,25993,25993,25993
mean,2.93366e+06,12595.9,0
std,1.66776e+06,86431.6,0
min,15,1001,0
25%,1.54103e+06,1511,0
50%,2.93997e+06,2760,0
75%,4.32818e+06,7177,0
max,5.9142e+06,6.81225e+06,0


In [10]:

import os.path
import numpy as np
from trust_score import TrustScore
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import json

y_pred = []
y_test=[]
cou = 0

for row in user_data.iterrows():
    path = 'user_data_50_95/rev_list_' + str(row[1]['id']) + '-d.json'
    #print(path)
    try:
        with open(path, 'r') as infile:
            data = json.loads(infile.read())

            if len(data) > 0:
                for d in data:
                    del d['next_rev']

                graph_for = "longevityRev"

                series = pd.DataFrame(data=data)
                series = series[['pageid', 'timestamp', graph_for]]
                series = series[series.longevityRev >= 0]
                if len(series) > 9:
                    print('User ID: ', str(row[1]['id']))
                    series['Trust'] = TrustScore([series[graph_for],48]).calculate()
                    series['longevityRevN'] = series['longevityRev'].shift(-1)
                    series['longevityRevP'] = series['longevityRev'].shift(1)
                    series = series[series.longevityRevP >= 0]
                    series = series[series.longevityRevN >= 0]
                    series = series[series.longevityRev >= 0]

                    index = 8
                    #print(series)
                    while (index < len(series)):
                        seriesTrain = series[0:index]
                        seriesTest = series[index:index+1]
#                         print(seriesTrain)
                        #print(seriesTest)
                        regressor = LinearRegression()
                        regressor.fit(seriesTrain[['Trust', 'longevityRev','longevityRevP']], seriesTrain['longevityRevN'])

                        pred = regressor.predict(seriesTest[['Trust', 'longevityRev','longevityRevP']])
                        print('Data Count: ', len(seriesTrain))
                        print('Test Data: ')
                        print(seriesTest[['Trust', 'longevityRev','longevityRevP']])
                        print('predicted: ',pred, ' actual: ', seriesTest['longevityRevN'].values[0])
                        print("##########################")
                        if(pred>=0):
                            y_test.append(seriesTest['longevityRevN'].values[0])
                            y_pred.append(pred)
                        index = index +1
    except Exception as e:
        print('error :')
        cou= cou+1

print(str(cou))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('R2 Score:', metrics.r2_score(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

User ID:  15
Data Count:  8
Test Data: 
    Trust  longevityRev  longevityRevP
17    0.0           0.0            0.0
predicted:  [0.]  actual:  0.0
##########################
Data Count:  9
Test Data: 
    Trust  longevityRev  longevityRevP
18    0.0           0.0            0.0
predicted:  [0.]  actual:  0.0
##########################
Data Count:  10
Test Data: 
    Trust  longevityRev  longevityRevP
19    0.0           0.0            0.0
predicted:  [0.]  actual:  0.0
##########################
Data Count:  11
Test Data: 
    Trust  longevityRev  longevityRevP
20    0.0           0.0            0.0
predicted:  [0.]  actual:  0.0
##########################
Data Count:  12
Test Data: 
    Trust  longevityRev  longevityRevP
21    0.0           0.0            0.0
predicted:  [0.]  actual:  0.0
##########################
Data Count:  13
Test Data: 
    Trust  longevityRev  longevityRevP
22    0.0           0.0            0.0
predicted:  [0.]  actual:  0.0
##########################
Data 